In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.appName("SQL").getOrCreate()

23/12/29 06:54:15 WARN Utils: Your hostname, DESKTOP-0KANSSR resolves to a loopback address: 127.0.1.1; using 192.168.228.28 instead (on interface eth0)
23/12/29 06:54:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/12/29 06:54:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
from pyspark.sql import Row
import random

## Criando DataFrame de clientes
schema_clientes = StructType([
    StructField('cliente_id', IntegerType(), True),
    StructField('nome', StringType(), True),
    StructField('idade', IntegerType(), True),
    StructField('cidade', StringType(), True),
    StructField('genero', StringType(), True)
])

clientes_data = [
    (i, 'Cliente' + str(i), random.randint(18, 65),
     random.choice(['São Paulo', 'Rio de Janeiro', 'Belo Horizonte', 'Brasília']),
     random.choice(['M', 'F']))
    for i in range(1, 1000)
]

df_clientes = spark.createDataFrame(clientes_data, schema=schema_clientes)

# Criando DataFrame de produtos
schema_produtos = StructType([
    StructField('produto_id', IntegerType(), True),
    StructField('nome_produto', StringType(), True),
    StructField('categoria', StringType(), True),
    StructField('preco', FloatType(), True)
   
])

produtos_data = [
    Row(
        produto_id=i,
        nome_produto='Produto' + str(i),
        categoria=random.choice(['Eletrônicos', 'Roupas', 'Alimentos', 'Acessórios']),
        preco=random.uniform(10, 1000),  # Convertendo para float
        disponibilidade=random.choice([True, False])
    )
    for i in range(1, 1000)
]

# Criando o DataFrame de produtos
df_produtos = spark.createDataFrame(produtos_data)

# # Criando DataFrame de vendas
schema_vendas = StructType([
    StructField('venda_id', IntegerType(), True),
    StructField('cliente_id', IntegerType(), True),
    StructField('produto_id', IntegerType(), True),
    StructField('quantidade', IntegerType(), True),
    StructField('data_venda', StringType(), True)
])

vendas_data = [
    (i, random.randint(1, 100), random.randint(1, 50), random.randint(1, 5), '2023-01-01')
    for i in range(1, 1000)
]

df_vendas = spark.createDataFrame(vendas_data, schema=schema_vendas)

# Criando DataFrame de transações
schema_transacoes = StructType([
    StructField('transacao_id', IntegerType(), True),
    StructField('produto_id', IntegerType(), True),
    StructField('valor', FloatType(), True),
    StructField('tipo', StringType(), True),
    StructField('status', StringType(), True)
])

transacoes_data = [
    (i, random.randint(1, 50), random.uniform(50, 500),
     random.choice(['Débito', 'Crédito']), random.choice(['Aprovado', 'Pendente', 'Rejeitado']))
    for i in range(1, 1000)
]

df_transacoes = spark.createDataFrame(transacoes_data, schema=schema_transacoes)


##### Group by with SUM and Case

In [40]:
df_transacoes.groupBy("tipo")\
    .agg(
        ((sum(when(col("status") == "Aprovado", 1).otherwise(0)) / 
         sum(when(col("status") == "Pendente", 1).otherwise(5)) * 100).cast("int").alias("teste") /100).alias("total")
    )\
    .show()

+-------+-----+
|   tipo|total|
+-------+-----+
|Crédito| 0.09|
| Débito| 0.08|
+-------+-----+



##### Group by with SUM and Case (Using EXPR)

In [41]:
df_transacoes.groupBy("tipo").agg(
    expr("CAST((sum(CASE WHEN status = 'Aprovado' THEN 1 ELSE 0 END) / sum(CASE WHEN status = 'Pendente' THEN 1 ELSE 5 END) * 100) AS INT) / 100 as Teste")
).show()

+-------+-----+
|   tipo|Teste|
+-------+-----+
|Crédito| 0.09|
| Débito| 0.08|
+-------+-----+



#### Join with where

In [10]:
df_vendas.alias("v")\
    .join(df_produtos.alias("p").filter("categoria = 'Alimentos'"), "produto_id")\
    .show(3)

+----------+--------+----------+----------+----------+------------+---------+----------------+---------------+
|produto_id|venda_id|cliente_id|quantidade|data_venda|nome_produto|categoria|           preco|disponibilidade|
+----------+--------+----------+----------+----------+------------+---------+----------------+---------------+
|        29|     990|        63|         4|2023-01-01|   Produto29|Alimentos|857.759464844174|           true|
|        29|     923|        20|         2|2023-01-01|   Produto29|Alimentos|857.759464844174|           true|
|        29|     803|        62|         3|2023-01-01|   Produto29|Alimentos|857.759464844174|           true|
+----------+--------+----------+----------+----------+------------+---------+----------------+---------------+
only showing top 3 rows

